In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
import IPython.display as ipd

import scipy.io as sio
import librosa

import time

import skimage
import skimage.io
from skimage.transform import rescale

from scipy.io import wavfile
import os
import subprocess

import sys
sys.path.append("../src")
from spectrogramtools import *
from stego import *
from swpowerstego import *
from imtools import *
from tsp import *
from curvature import *

In [ ]:
# Image parameters
thresh = 0.1
p = 1
canny_sigma = 3
density_fac= 1
edge_weight = 10
target_points = 3000

np.random.seed(0)
I = skimage.io.imread("../data/images/usher.jpg")

X = voronoi_stipple(I, thresh, target_points=target_points, p=p, canny_sigma=canny_sigma, edge_weight=edge_weight, n_iters=10, do_plot=False)
plt.figure(figsize=(10, 10))
plt.scatter(X[:, 0], X[:, 1])

In [ ]:
#X = density_filter(X, fac=density_fac, k=2)
X = get_tsp_tour(X)

plt.figure(figsize=(16, 8))
plt.subplot(121)
plt.plot(X[:, 0], X[:, 1])
plt.scatter(X[:, 0], X[:, 1])

## Load in audio and compute best target

In [ ]:
# Audio parameters
x, sr = librosa.load("../data/tunes/barry.mp3", sr=44100)
win_length = 512

SX = stft_disjoint(x, win_length)
SXM = np.abs(SX)
plt.plot(np.sum(SXM, axis=1))

In [ ]:
# Sliding window parameters
win = 800
fit_lam = 1
q = -1
min_freq = 100
max_freq = 8000

sp = STFTPowerDisjointPCA(x, X, sr, win_length, min_freq, max_freq, win, fit_lam, q)


plt.figure()
plt.plot(sp.path)

sp.plot()

sp.solve(use_constraints=False)
sp.plot(normalize=True)
y = sp.reconstruct_signal()
print("SNR", get_snr(x, y))
ipd.Audio(y, rate=sr)

In [ ]:
prefix = "SpecPCA_Win{}_fit{:.3g}_stftwin{}_q{}_fmin{}_fmax{}".format(win, fit_lam, win_length, q, min_freq, max_freq)
filename = "{}.mp3".format(prefix)
wavfile.write("out.wav", sr, y)
if os.path.exists(filename):
    os.remove(filename)
subprocess.call("ffmpeg -i out.wav".split() + [filename], stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT)
os.remove("out.wav")

z, sr = librosa.load(filename, sr=sr)
#z = z[100::]
z_sp = STFTPowerDisjointPCA(z, X, sr, win_length, min_freq, max_freq, win, fit_lam, q)
z_sp.targets = sp.targets
snr = get_snr(x, z)
distortion = z_sp.get_distortion()[0]
ratio = snr/distortion

title = "snr = {:.3f}, distortion = {:.3f}".format(snr, distortion)

plt.figure(figsize=(6, 6))
z_sp.plot(True, title)
plt.tight_layout()
plt.savefig("{}.png".format(prefix), facecolor='white')



In [ ]:
"""
from scipy.spatial import ConvexHull, convex_hull_plot_2d

Z = z_sp.get_signal(True)
#Z[:, 1] *= -1

s = get_arclen(get_curv_vectors(Z, 0, 1, loop=True)[1])
Z = arclen_resample(Z, s, Z.shape[0])
sigma = 0.5
Z = get_curv_vectors(Z, 0, sigma, loop=True)[0]


hull = ConvexHull(Z)
ZHull = Z[hull.vertices, :]

plt.figure(figsize=(10, 10))
plt.scatter(Z[:, 0], Z[:, 1])
plt.plot(Z[:, 0], Z[:, 1])
plt.plot(ZHull[:, 0], ZHull[:, 1])
plt.axis("equal")
"""

In [ ]:
Y = z_sp.get_target(normalize=True)
Y = Y - np.mean(Y, axis=0)[None, :]


ZRot, min_d = z_sp.get_transformed_distortion()
print(min_d)
plt.figure(figsize=(16, 8))
plt.subplot(121)
plt.scatter(Y[:, 0], Y[:, 1])
plt.axis("equal")
plt.title("Length = {:.3f}".format(get_length(Y)))
plt.subplot(122)
plt.scatter(ZRot[:, 0], ZRot[:, 1])
plt.axis("equal")
plt.title("Length = {:.3f}".format(get_length(ZRot)))